# Análisis Breve de Coordinación de Protecciones

Los datos corresponden al análisis de coordinación de protecciones del escenario "scenario_1". El sistema cuenta con 37 líneas, cada una equipada con exactamente 2 relés (sin variación), y 5 mallas. Los parámetros de coordinación muestran que el "Pickup" tiene un promedio de 0.06 con baja dispersión, mientras que "Ishc" (corriente de cortocircuito) varía ampliamente (media de 1.79, máximo de 10.23). El "TDS" (ajuste de tiempo) y "Time_out" presentan valores medios de 0.10 y 0.21, respectivamente, con rangos moderados. Esto sugiere un sistema bien coordinado, con relés consistentemente distribuidos y parámetros ajustados, aunque con algunas corrientes de cortocircuito significativamente altas.

In [1]:
import json
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the JSON file
file_path = Path("/Users/gustavo/Documents/Projects/TESIS_UNAL/optimizacion-coordinacion-protecciones/data/raw/data_coordination_scenario_base.json")
with open(file_path, 'r') as file:
    data = json.load(file)

def analyze_coordination_data(data):
    """Analyze coordination data and generate interactive visualizations."""
    
    # 1. Basic Information
    print("=== Basic Information ===")
    print(f"Scenario: {data['scenario_id']}")
    print(f"Timestamp: {data['timestamp']}")
    print(f"Number of lines: {len(data['base_data'])}")
    print(f"Number of meshes: {len(data['meshes'])}")

    # 2. Relay Analysis per Line
    relay_counts = {line: len(line_data['relays']) 
                   for line, line_data in data['base_data'].items()}
    
    relay_df = pd.DataFrame.from_dict(relay_counts, orient='index', 
                                    columns=['Number of Relays'])
    print("\n=== Relay Statistics per Line ===")
    print(relay_df.describe().round(2))

    # 3. Coordination Parameters Analysis
    coordination_data = []

    for line, scenarios in data['results_by_line'].items():
        for scenario, details in scenarios['scenarios'].items():
            # Main relay
            main = details['main']
            main_entry = {
                'Line': line,
                'Scenario': scenario,
                'Type': 'Main',
                'Relay': main['relay'],
                'Pickup': main['pick_up'],
                'Ishc': main['Ishc'],
                'TDS': None,
                'Time_out': None
            }
            if main.get('coordination_history'):
                for history in main['coordination_history']:
                    entry = main_entry.copy()
                    entry.update({
                        'TDS': history['TDS'],
                        'Time_out': history['Time_out'],
                        'History_Timestamp': history['timestamp']
                    })
                    coordination_data.append(entry)
            else:
                coordination_data.append(main_entry)

            # Backup relays
            for backup in details['backups']:
                backup_entry = {
                    'Line': line,
                    'Scenario': scenario,
                    'Type': 'Backup',
                    'Relay': backup['relay'],
                    'Pickup': backup['pick_up'],
                    'Ishc': backup['Ishc'],
                    'TDS': None,
                    'Time_out': None
                }
                if backup.get('coordination_history'):
                    for history in backup['coordination_history']:
                        entry = backup_entry.copy()
                        entry.update({
                            'TDS': history['TDS'],
                            'Time_out': history['Time_out'],
                            'History_Timestamp': history['timestamp']
                        })
                        coordination_data.append(entry)
                else:
                    coordination_data.append(backup_entry)

    # Create DataFrame
    stats_df = pd.DataFrame(coordination_data)
    print("\n=== Coordination Parameters Statistics ===")
    print(stats_df.describe().round(2))

    # 4. Interactive Visualizations with Plotly
    fig = make_subplots(rows=2, cols=2,
                       subplot_titles=('Pickup Distribution', 
                                     'Ishc Distribution',
                                     'TDS Distribution',
                                     'Time_out Distribution'))

    # Pickup Histogram
    fig.add_trace(
        go.Histogram(x=stats_df['Pickup'].dropna(), 
                    name='Pickup',
                    marker_color='#1f77b4'),
        row=1, col=1
    )

    # Ishc Histogram
    fig.add_trace(
        go.Histogram(x=stats_df['Ishc'].dropna(), 
                    name='Ishc',
                    marker_color='#ff7f0e'),
        row=1, col=2
    )

    # TDS Histogram
    fig.add_trace(
        go.Histogram(x=stats_df['TDS'].dropna(), 
                    name='TDS',
                    marker_color='#2ca02c'),
        row=2, col=1
    )

    # Time_out Histogram
    fig.add_trace(
        go.Histogram(x=stats_df['Time_out'].dropna(), 
                    name='Time_out',
                    marker_color='#d62728'),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        height=800,
        width=1000,
        title_text="Distribution of Coordination Parameters",
        showlegend=False,
        bargap=0.1
    )

    # Update axes
    fig.update_xaxes(title_text="Pickup", row=1, col=1)
    fig.update_xaxes(title_text="Ishc", row=1, col=2)
    fig.update_xaxes(title_text="TDS", row=2, col=1)
    fig.update_xaxes(title_text="Time_out", row=2, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    fig.update_yaxes(title_text="Frequency", row=2, col=2)

    fig.show()

    # 5. Mesh Analysis
    print("\n=== Mesh Analysis ===")
    for mesh_id, mesh_data in data['meshes'].items():
        print(f"\nMesh: {mesh_id}")
        print(f"Number of nodes: {len(mesh_data['path'])}")
        print(f"First iteration timestamp: {mesh_data['iterations'][0]['timestamp']}")
        print(f"Description: {mesh_data['iterations'][0]['description']}")

    # Additional Analysis: Relay Type Distribution
    type_dist = stats_df['Type'].value_counts()
    fig2 = px.pie(values=type_dist.values, 
                 names=type_dist.index,
                 title='Distribution of Relay Types')
    fig2.show()

if __name__ == "__main__":
    analyze_coordination_data(data)

=== Basic Information ===
Scenario: scenario_1
Timestamp: 2025-04-08T17:33:09.220228
Number of lines: 37
Number of meshes: 5

=== Relay Statistics per Line ===
       Number of Relays
count              37.0
mean                2.0
std                 0.0
min                 2.0
25%                 2.0
50%                 2.0
75%                 2.0
max                 2.0

=== Coordination Parameters Statistics ===
       Pickup    Ishc     TDS  Time_out
count  817.00  805.00  805.00    805.00
mean     0.06    1.79    0.10      0.21
std      0.04    1.78    0.05      0.10
min      0.01    0.27    0.05      0.06
25%      0.03    0.84    0.05      0.10
50%      0.06    1.27    0.09      0.27
75%      0.07    2.00    0.14      0.29
max      0.21   10.23    0.22      0.42



=== Mesh Analysis ===

Mesh: mesh1
Number of nodes: 29
First iteration timestamp: 2025-04-08T17:33:09.954641
Description: Coordinación optimizada para TMT cercano a -0.9

Mesh: mesh2
Number of nodes: 16
First iteration timestamp: 2025-04-08T17:33:10.574486
Description: Coordinación optimizada para TMT cercano a -0.9

Mesh: mesh3
Number of nodes: 24
First iteration timestamp: 2025-04-08T17:33:11.163952
Description: Coordinación optimizada para TMT cercano a -0.9

Mesh: mesh4
Number of nodes: 28
First iteration timestamp: 2025-04-08T17:33:11.648118
Description: Coordinación optimizada para TMT cercano a -0.9

Mesh: mesh5
Number of nodes: 24
First iteration timestamp: 2025-04-08T17:33:12.147058
Description: Coordinación optimizada para TMT cercano a -0.9
